In [1]:
import polars as pl
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars

id.orig_p,id.resp_p,conn_state,history,orig_ip_bytes,label
i32,i32,i64,i64,i64,i32
5526,37215,0,0,40,1
60403,23,2,7,40,1
13386,81,2,7,40,1
36097,37215,0,0,40,1
36097,37215,0,0,40,1
…,…,…,…,…,…
30535,8081,1,1,80,1
36097,37215,0,0,40,1
41258,23,1,1,120,1


In [9]:
df_polars = df_polars.drop_nulls()

In [10]:
X = df_polars.drop('label')
y = df_polars['label']       

In [11]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [12]:
results = []

In [13]:
i=5
knn = KNeighborsClassifier(
n_neighbors=i,
metric='euclidean', 
weights='uniform'  
)

In [ ]:
def startTrain(X, y, n_neighbors, metric, weights):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Aplicando a normalização APENAS nos dados de treino
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Aplicando o undersampling no conjunto de treino
        rus = RandomUnderSampler()
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

        # Criando e treinando o modelo KNN
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
        
        start_training = time.time() 
        knn.fit(X_train_resampled, y_train_resampled)
        end_training = time.time()

        # Predição e avaliação
        y_pred = knn.predict(X_test_scaled)
        evaluation_duration = time.time() - end_training
        training_duration = end_training - start_training

        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp)
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["KNN"] + mean_results.tolist())

In [15]:
#Melhores Hiperparâmetros encontrados: {'n_neighbors': 3, 'metric': 'manhattan', 'weights': 'distance'}

In [16]:
for i in range(1,11):
    startTrain(X=X, y=y, n_neighbors=3, metric='manhattan', weights='distance')
    print(i)

1
2
3
4
5
6
7
8
9
10


In [17]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy' , 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""KNN""",0.992074,0.994339,0.999552,0.991004,0.997674,0.99526,0.002326,17761.800781,41.400002,838.799988,92404.601562,0.211882,9.701704
"""KNN""",0.992113,0.994437,0.999587,0.991017,0.997856,0.995283,0.002144,17765.0,38.200001,837.599976,92405.796875,0.209319,9.220567
"""KNN""",0.992122,0.994428,0.99958,0.991035,0.997822,0.995289,0.002178,17764.400391,38.799999,836.0,92407.398438,0.206737,9.149642
"""KNN""",0.992129,0.994504,0.999615,0.991008,0.998,0.995293,0.002,17767.599609,35.599998,838.400024,92405.0,0.206783,9.156576
"""KNN""",0.99213,0.994463,0.999595,0.991028,0.997899,0.995293,0.002101,17765.800781,37.400002,836.599976,92406.796875,0.205704,9.126111
"""KNN""",0.99214,0.994352,0.999539,0.991096,0.997608,0.9953,0.002392,17760.599609,42.599998,830.200012,92413.203125,0.209637,9.105722
"""KNN""",0.992144,0.994455,0.999587,0.991054,0.997857,0.995302,0.002143,17765.0,38.200001,834.200012,92409.203125,0.206263,9.189205
"""KNN""",0.992155,0.994465,0.999589,0.991064,0.997866,0.995308,0.002134,17765.199219,38.0,833.200012,92410.203125,0.207585,9.592356
"""KNN""",0.992095,0.994416,0.999582,0.991,0.997832,0.995273,0.002168,17764.599609,38.599998,839.200012,92404.203125,0.219124,10.561996


In [ ]:
metrics_df.write_csv(f"metrics_results/balanced_knn_metrics_output.csv", separator=';')